In [ ]:
from scipy.spatial.transform import Rotation
import numpy as np

# Rotations
How do we move measurements between reference frames? With rotations!

Let's look at a few ways to define rotations in 3D.

# Euler angles

<img src="figures/Eulerangles.svg" alt="How Euler Angles Are Defined" align="left" style="width: 300px;"/>

Euler angles define rotations by listing three rotations about specific axes. If you've ever heard of roll, pitch, and yawn then you've heard about Euler angles. One nice property of Euler angles is that they describe the rotation in a way that a human can easily recreate. Euler angles are an algorithmic way to define rotations.

Unfortunately, Euler angles also come with a lot of [drawbacks](https://github.com/moble/quaternion/wiki/Euler-angles-are-horrible). Rotations are non-commutative, which means the order you do them in matters. So, if you rotate about the X-axis by 90 degrees and then the Z-axis by 90 degrees, you will get a different rotation than if you rotate about the Z-axis by 90 degrees and then the X-axis by 90 degrees. Euler angles are a series of rotations about axes, but there is no convention for what order you rotate around the axes in, or even which axes you rotate about. **If you're working with Euler angles always be aware of what your axis order is!**

In [ ]:
x_axis = np.array([1, 0, 0])
y_axis = np.array([0, 1, 0])
z_axis = np.array([0, 0, 1])

sensor_to_target = Rotation.from_euler('xyz', np.array([90, 0, 90]), degrees=True)

print('The x axis rotates to', sensor_to_target.apply(x_axis))
print('The y axis rotates to', sensor_to_target.apply(y_axis))
print('The z axis rotates to', sensor_to_target.apply(z_axis))

## Rotation Matrices

<img src="figures/rotation_matrices.svg" alt="The three fundamental rotation matrices" align="left" style="width: 300px;"/>

Rotation matrices define rotations as functions that map between the coordinate systems. Rotation matrices are a very common way to represent rotations in mathematics both because they are easy to work with, only requiring simple linear algebra, and because they make great examples in group theory.

Rotation matrices make manually rotationing things quite simple. All you have to do to rotate a vector is left multiply it by the rotation matrix.

In [ ]:
# scipy calls the rotation matrix the DCM because what people commonly call a rotation matrix is
# specifically a Directed Cosine Matrix representation of the rotation
print('Our rotation as a rotation matrix is:\n', sensor_to_target.as_dcm())

<span style="color:blue">Write a code snippet to manually rotate the X, Y, and Z axes by our rotation using its rotation matrix.</span>

In [ ]:
# You can use np.dot(M, x) to left multiply a vector, x, by a matrix, M.


Rotation matrices also make applying multiple rotations simple! To apply two rotations to a vector, you simply left multiply by another rotation matrix.

<span style="color:blue">Using rotations about individual axes and matrix multiplication re-create the rotation we defined using Euler angles.</span>

In [ ]:
# You can create a rotation about a single axis using Rotation.from_euler('x', 90, degrees=True)


Rotation matrices are convenient if you have what to see how a rotation affects specific vectors, but they are quite large. Compared to Euler angles, you have to store 9 floating point numbers instead of just three. Additionally, performing matrix multiplication requires [more operations](https://en.wikipedia.org/wiki/Quaternions_and_spatial_rotation#Performance_comparisons) than other ways of chaining rotations.

## Quaternions

<img src="figures/Broom_bridge_plaque.jpg" alt="The plaque on Broom bridge where Hamilton supposedly came up with the idea of using quaternions to represent rotations" align="left" style="width: 600px;"/>

Quaternions are a complicated way to represent rotations, but it turns out they have a lot of really useful properties.

### What are quaternions?
If you don't know what quaternions are, then you can think about them as complex numbers except instead of just i, you also get j and k. For example, $1+0.5i+2j+3k$ is a quaternion, so is $3+0i+2.3j+4k$. When representation rotations, unit quaternions are usually used. This is because scaling a quaternion by a constant doesn't change the rotation it represents. So, $1+0.5i+2j+3k$ represents the same rotation as $2+1i+4j+6k$.

### How do quaternions represent rotations?
Due to some interesting properties of [quaternion maultiplication](http://graphics.stanford.edu/courses/cs348a-17-winter/Papers/quaternion.pdf), we can represent a rotation around a unit length axis $(x,y,z)$ by $\theta$ with the following quaternion $\cos\left(\frac{\theta}{2}\right) + \sin\left(\frac{\theta}{2}\right)(xi + yj + zk)$.

That's nifty and all but how do we actually use it? Let's say we have some quaternion $q$ that represents a rotation and we want to rotate a vector $(a,b,c)$ by that rotation. The way we do that is by first converting $(a,b,c)$ into the pure quaternion $0+ai+bj+ck$, then we multiply it on the left by $q$ and on the right by $q^{-1}$, the inverse of $q$. This produces another pure quaternion $0+a^\prime i+b^\prime j+c^\prime k = q(0+ai+bj+ck)q^{-1}$ and our rotated vector is simply $(a^\prime,b^\prime,c^\prime)$. Computing the right side of that equals sign by hand can be rather tedious, but thankfully it's just a bit of addition and multiplication, so computers can do it quite quickly.

### What are some other perks of quaternions?
* Computing a rotation matrices from a quaternion requires **0** trig functions.
* Every rotation is represented by exactly 2 quaternions, $q$ and $-q$.
* The space of unit quaternions is a 4 dimensional sphere, so they interpolate nicely.
* They are relatively compact, only 4 floating point values need to be stored.
* They chain together easily; to rotate by two quaternions, you simply rotate by the product of the quaternions.

### What are some problems with quaternions?
* They are abstract; it is hard to intuitively understand how a quaternion rotates a given vector.
* Rotating a vector by a quaternion takes more operations than rotating a vector by a rotation matrix.
* Libraries are inconsistent about ordering quaternions; some use $w +xi + yj + zk$ (SPICE) and others use $xi + yj + zk + w$ (scipy)

### Wow that was a lot
Yes it was, let's actually use some quaternions to do rotations.

In [ ]:
print('Our rotation as a quaternion is', sensor_to_target.as_quat())

<span style="color:blue">What is a 90 degree rotation around the X-axis as a quaternion?</span>

In [ ]:
# Note that scipy orders their quaternions as xi + yj + zk + w, so the no rotation quaternion 1 + 0i + 0j + 0k is [0, 0, 0, 1]
quat = np.array([ , , , ])
# check your self
assert np.linalg.norm(Rotation.from_euler('x', 90, degrees=True).as_quat() - quat) < 1e-15

<span style="color:blue">What are the magnitude and axis of our rotation?</span>

In [ ]:
# Hint use cos(theta/2) + sin(theta/2)(xi + yj + zk)
